### 네이버 위도 경도 뽑기


In [ ]:
import requests
import pandas as pd
import time

In [ ]:
XY = pd.read_csv('101-600.csv')
XY_unique = XY.drop_duplicates(subset='tourist_spot')
XY_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, 0 to 3099
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tourist_spot     187 non-null    object
 1   good_point       187 non-null    object
 2   selected_people  187 non-null    int64 
 3   addr             46 non-null     object
 4   total_people     46 non-null     object
dtypes: int64(1), object(4)
memory usage: 8.8+ KB


In [ ]:
import requests
import pandas as pd
import time

# 네이버 API 키 설정
client_id = 'client_id 넣어주세요 '
client_secret = 'client_secret key 넣어주세요 '

def find_places_naver(keywords):
    success_data = []
    fail_data = []

    for searching in keywords:
        try:
            url = f'https://openapi.naver.com/v1/search/local.json?query={searching}&display=1'
            headers = {
                "X-Naver-Client-Id": client_id,
                "X-Naver-Client-Secret": client_secret
            }

            response = requests.get(url, headers=headers)
            places = response.json().get('items', [])

            # 오류 및 실패 처리..
            if not places:
                fail_data.append((searching, 'No data'))
                time.sleep(1)
                continue

            place = places[0]
            name = place['title']

            # 정수로 가져와서 소수점 맞춰줌
            x = float(place['mapx']) / 10000000  # 소수점 포함 형식으로 변환
            y = float(place['mapy']) / 10000000  # 소수점 포함 형식으로 변환
            success_data.append((name, x, y, searching))

        except Exception as e:
            fail_data.append((searching, str(e)))

        # 각 요청 사이에 1초 대기
        time.sleep(1)

    success_df = pd.DataFrame(success_data, columns=['Name', 'Longitude', 'Latitude', 'attraction'])
    fail_df = pd.DataFrame(fail_data, columns=['Search Term', 'Error'])

    return success_df, fail_df

# --------------------------------------------------------------------------------------------------------



# csv 여기다가 불러오기
XY = pd.read_csv('101-600.csv')
XY_unique = XY.drop_duplicates(subset='tourist_spot')

# 리스트로 변환
Location_name_list = XY_unique['tourist_spot'].tolist()

# 빈 데이터프레임에 컬럼명 생성
all_success_df = pd.DataFrame(columns=['Name', 'Longitude', 'Latitude', 'attraction'])
all_fail_df = pd.DataFrame(columns=['Search Term', 'Error'])

# 각 관광지에 대해 검색을 수행
for name in Location_name_list:
    keywords = [name]  # 리스트로 감싸서 단일 키워드로 맨듬
    success_df, fail_df = find_places_naver(keywords)

    all_success_df = pd.concat([all_success_df, success_df], ignore_index=True)
    all_fail_df = pd.concat([all_fail_df, fail_df], ignore_index=True)

# 성공한 데이터프레임에서 중복 제거 후 첫번째 한 개만 출력
XY_unique_results = all_success_df.drop_duplicates(subset='Name', keep='first')
XY_unique_results = XY_unique_results[['Longitude', 'Latitude', 'attraction']].copy()


# 결과 출력
print("성공한 데이터프레임 (중복 제거 후):")
print(XY_unique_results)

print("\n실패한 데이터프레임:")
print(all_fail_df)

C:\Users\taeyoung\AppData\Local\Temp\ipykernel_23628\1002234001.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_success_df = pd.concat([all_success_df, success_df], ignore_index=True)
C:\Users\taeyoung\AppData\Local\Temp\ipykernel_23628\1002234001.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_success_df = pd.concat([all_success_df, success_df], ignore_index=True)
C:\Users\taeyoung\AppData\Local\Temp\ipykernel_23628\1002234001.py:69: FutureWarning: The behavior of DataFrame co

성공한 데이터프레임 (중복 제거 후):
      Longitude   Latitude              attraction
0    127.080921  37.549899                서울어린이대공원
1    127.043502  37.546288                섬세이 테라리움
2    126.963300  37.589021               목인박물관 목석원
3    127.090589  37.573197                  용마폭포공원
4    126.978931  37.581564  국립민속박물관&국립민속박물관 어린이박물관
..          ...        ...                     ...
180  127.105360  37.594959                    용마랜드
181  126.985803  37.573485                 통인가게 미술
182  127.098164  37.511079                    롯데월드
183  126.990628  37.498917             서초구민체육센터수영장
184  126.969839  37.580057                서촌게스트하우스

[185 rows x 3 columns]

실패한 데이터프레임:
      Search Term    Error
0    영등포 신길동 홍어거리  No data
1  세종한우정육점식당  2호점  No data


In [ ]:
# 실패 데이터 처리 완료

### 저장 및 확인

In [ ]:
XY_unique_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Longitude   62 non-null     float64
 1   Latitude    62 non-null     float64
 2   attraction  62 non-null     object 
dtypes: float64(2), object(1)
memory usage: 1.6+ KB


In [ ]:
all_fail_df

,Search Term,Error
0,영등포 신길동 홍어거리,No data
1,세종한우정육점식당 2호점,No data


In [ ]:
XY_unique_results.to_csv('test.csv', encoding='cp949')